In [1]:
import os
from os.path import exists, join, expanduser

# Clone
name = "deepvoice3_pytorch"
if not exists(name):
  ! git clone https://github.com/r9y9/$name
else:
    print("Exists!")

Exists!


In [2]:
# Change working directory to the project dir 
os.chdir(join(expanduser("."), name))

# Install dependencices
# !pip install -q -e '.[train]'

In [3]:
import torch
import numpy as np
import librosa
import librosa.display
import IPython
from IPython.display import Audio
# need this for English text processing frontend
import nltk
! python3 -m nltk.downloader cmudict

[nltk_data] Downloading package cmudict to /home/mehul/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [4]:
checkpoint_path = "20171222_deepvoice3_vctk108_checkpoint_step000300000.pth"

In [5]:
if not exists(checkpoint_path):
    pass
#   !curl -O -L "https://www.dropbox.com/s/uzmtzgcedyu531k/20171222_deepvoice3_vctk108_checkpoint_step000300000.pth"

In [6]:
# Copy preset file (json) from master
# The preset file describes hyper parameters
# ! git checkout master --quiet
preset = "./presets/deepvoice3_vctk.json"
# ! cp -v $preset .
# preset = "./deepvoice3_vctk.json"

# And then git checkout to the working commit
# This is due to the model was trained a few months ago and it's not compatible
# with the current master. 
# ! git checkout 0421749 --quiet
# ! pip install -q -e .

In [7]:
! git branch

* master


In [8]:
print(os.getcwd())
import hparams
import json
# print(hparams.hparams.get_model_structure())
# Newly added params. Need to inject dummy values
for dummy, v in [("fmin", 0), ("fmax", 0), ("rescaling", False),
                 ("rescaling_max", 0.999), 
                 ("allow_clipping_in_normalization", False)]:
  #if hparams.hparams.get(dummy) is None:
    hparams.hparams.add_hparam(dummy, v)
    
# Load parameters from preset
with open(preset) as f:
  hparams.hparams.parse_json(f.read())

# Tell we are using multi-speaker DeepVoice3
hparams.hparams.builder = "deepvoice3_multispeaker"
  
# Inject frontend text processor
import synthesis
import train
from deepvoice3_pytorch import frontend
synthesis._frontend = getattr(frontend, "en")
train._frontend =  getattr(frontend, "en")



/home/mehul/machine_learning/Speech Processing/Neural-Voice-Cloning-with-Few-Samples/deepvoice3_pytorch


In [18]:
def tts(model, text, p=0, speaker_id=0, fast=True, figures=True):
  from synthesis import tts as _tts
  waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
  if figures:
      visualize(alignment, spectrogram)
  IPython.display.display(Audio(waveform, rate=fs))
  
def visualize(alignment, spectrogram):
  label_fontsize = 16
  figure(figsize=(16,16))

  subplot(2,1,1)
  imshow(alignment.T, aspect="auto", origin="lower", interpolation=None)
  xlabel("Decoder timestamp", fontsize=label_fontsize)
  ylabel("Encoder timestamp", fontsize=label_fontsize)
  colorbar()

  subplot(2,1,2)
  librosa.display.specshow(spectrogram.T, sr=fs, 
                           hop_length=hop_length, x_axis="time", y_axis="linear")
  xlabel("Time", fontsize=label_fontsize)
  ylabel("Hz", fontsize=label_fontsize)
  tight_layout()
  colorbar()

In [9]:
from train import build_model
from train import restore_parts, load_checkpoint

model = build_model()
# model = load_checkpoint(checkpoint_path, model, None, True)

In [ ]:
# Try your favorite senteneces:)
text = "sharad is not here"
N = 15
print("Synthesizing \"{}\" with {} different speakers".format(text, N))
for speaker_id in range(N):
  print(speaker_id)
  tts(model, text, speaker_id=speaker_id, figures=False)

In [ ]:
# With attention plot
tts(model, text, speaker_id=0, figures=True)

In [ ]:
text = "here i am"
speaker_id = 0
fast = True
p = 0
waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
print(waveform.shape)
print(alignment.shape)
print(spectrogram.shape)
print(mel.shape)
print(type(mel))

In [ ]:
cloning_texts = ["this is the first" , "this is the first"]
N = 2
p = 0
fast = True

In [ ]:
from synthesis import tts as _tts
import numpy as np


all_speakers = []
for speaker_id in range(N):
    speaker_cloning_mel = []
    print("The Speaker being cloned speaker-{}".format(speaker_id))
    for text in cloning_texts:
        waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
        speaker_cloning_mel.append(mel)
        #print(np.array(speaker_cloning_mel).shape)
    all_speakers.append(speaker_cloning_mel)

np.array(all_speakers).shape
# print(all_speakers.shape)

In [ ]:
# all speakers[speaker_id][cloned_audio_number]
all_speakers[0][1].shape

In [ ]:
import pickle 


with open("speaker_cloned_voices.txt", "wb") as fp:   #Pickling
    pickle.dump(all_speakers, fp)

In [5]:
try:
    with open("speaker_cloned_voices.txt" , "rb") as fp:
        b = pickle.load(fp)
except:
    b = None

In [12]:
# if b == None:
    #run deep voice to get cloned voices of 108 differentspeakers
#     pass
    

In [13]:
model.state_dict()

OrderedDict([('seq2seq.encoder.embed_tokens.weight', 
              -4.5694e-02  5.2645e-03 -1.5689e-01  ...  -5.5819e-02 -7.0906e-02  9.6754e-03
              -3.9518e-02 -1.1647e-01 -7.1957e-02  ...   4.0104e-02  8.5220e-02 -5.0006e-02
               6.9203e-02  7.3925e-02  3.2925e-02  ...  -5.7554e-02 -1.1573e-01 -2.3808e-01
                              ...                   ⋱                   ...                
              -1.4920e-01  4.7236e-02  1.3181e-01  ...   3.4376e-02  9.4757e-02 -5.4313e-02
              -1.2264e-02 -8.4443e-02 -7.1826e-02  ...   2.2476e-01 -1.7180e-01  3.8906e-02
              -9.1584e-02 -9.0224e-02 -7.4653e-02  ...   4.3111e-02  3.8053e-02  8.1325e-02
              [torch.FloatTensor of size 149x256]),
             ('seq2seq.encoder.speaker_fc1.bias', 
               0
               0
               0
               0
               0
               0
               0
               0
               0
               0
               0
            

In [20]:
for name,value in torch.nn.Module.named_parameters(model):
    print(name)

seq2seq.encoder.embed_tokens.weight
seq2seq.encoder.speaker_fc1.bias
seq2seq.encoder.speaker_fc1.weight_g
seq2seq.encoder.speaker_fc1.weight_v
seq2seq.encoder.speaker_fc2.bias
seq2seq.encoder.speaker_fc2.weight_g
seq2seq.encoder.speaker_fc2.weight_v
seq2seq.encoder.convolutions.0.bias
seq2seq.encoder.convolutions.0.weight_g
seq2seq.encoder.convolutions.0.weight_v
seq2seq.encoder.convolutions.2.conv.bias
seq2seq.encoder.convolutions.2.conv.weight_g
seq2seq.encoder.convolutions.2.conv.weight_v
seq2seq.encoder.convolutions.2.speaker_proj.bias
seq2seq.encoder.convolutions.2.speaker_proj.weight_g
seq2seq.encoder.convolutions.2.speaker_proj.weight_v
seq2seq.encoder.convolutions.3.conv.bias
seq2seq.encoder.convolutions.3.conv.weight_g
seq2seq.encoder.convolutions.3.conv.weight_v
seq2seq.encoder.convolutions.3.speaker_proj.bias
seq2seq.encoder.convolutions.3.speaker_proj.weight_g
seq2seq.encoder.convolutions.3.speaker_proj.weight_v
seq2seq.encoder.convolutions.4.conv.bias
seq2seq.encoder.convo

In [23]:
model.embed_speakers.weight.data.shape

torch.Size([108, 16])

In [25]:
model.seq2seq.encoder.embed_tokens.weight.data.shape

torch.Size([149, 256])